# 改变对象的字符串显示

`!r` - convert the value to a string using `repr()`.  
`!s` - convert the value to a string using `str()`.

`[[fill]align][sign][#][0][minimumwidth][.precision][type]`

In [4]:
class Pair:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return 'Pair({0.x!r}, {0.y!r})'.format(self)

    def __str__(self):
        return '({0.x!s}, {0.y!s})'.format(self)
p = Pair(3,4)
p,print(p)

(3, 4)


(Pair(3, 4), None)

In [5]:
print('p is {0!r}'.format(p)),print('p is {0!s}'.format(p))

p is Pair(3, 4)
p is (3, 4)


(None, None)

`__repr__()` 生成的文本字符串标准做法是需要让 `eval(repr(x)) == x`

# 自定义字符串的格式化 

In [6]:
_formats = {
    'ymd' : '{d.year}-{d.month}-{d.day}',
    'mdy' : '{d.month}/{d.day}/{d.year}',
    'dmy' : '{d.day}/{d.month}/{d.year}'
    }

class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

    def __format__(self, code):
        if code == '':
            code = 'ymd'
        fmt = _formats[code]
        return fmt.format(d=self)

In [7]:
d = Date(2012,12,21)
format(d)

'2012-12-21'

In [8]:
format(d,'mdy')

'12/21/2012'

In [9]:
'The date is {:ymd}'.format(d),'The date is {:mdy}'.format(d)

('The date is 2012-12-21', 'The date is 12/21/2012')

# 让对象支持上下文管理协议

In [10]:
from socket import socket, AF_INET, SOCK_STREAM

class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = family
        self.type = type
        self.sock = None

    def __enter__(self):
        if self.sock is not None:
            raise RuntimeError('Already connected')
        self.sock = socket(self.family, self.type)
        self.sock.connect(self.address)
        return self.sock

    def __exit__(self, exc_ty, exc_val, tb):
        self.sock.close()
        self.sock = None

In [13]:
from functools import partial

conn = LazyConnection(('www.python.org', 80))
# Connection closed
with conn as s:
    # conn.__enter__() executes: connection open
    s.send(b'GET /index.html HTTP/1.0\r\n')
    s.send(b'Host: www.python.org\r\n')
    s.send(b'\r\n')
    resp = b' '.join(iter(partial(s.recv, 8192), b''))
    print(resp.decode())
    # conn.__exit__() executes: connection closed

HTTP/1.1 301 Moved Permanently
Server: Varnish
Retry-After: 0
Location: https://www.python.org/index.html
Content-Length: 0
Accept-Ranges: bytes
Date: Mon, 23 Aug 2021 06:12:30 GMT
Via: 1.1 varnish
Connection: close
X-Served-By: cache-tyo11936-TYO
X-Cache: HIT
X-Cache-Hits: 0
X-Timer: S1629699151.693887,VS0,VE0
Strict-Transport-Security: max-age=63072000; includeSubDomains




# 创建大量对象是节省内存

类添加 `__slots__` 属性来极大的减少实例所占的内存  
当你定义 `__slots__` 后，Python就会为实例使用一种更加紧凑的内部表示

In [14]:
class Date:
    __slots__ = ['year', 'month', 'day']
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

# 在类中封装属性名

第一个约定是任何以单下划线`_`开头的名字都应该是内部实现。  
双下划线`__`开始会导致访问名称变成其他形式,继承——这种属性通过继承是无法被覆盖的  
定义的一个变量和某个保留关键字冲突，这时候可以使用单下划线`_`作为后缀

# 创建可管理的属性

In [15]:
class Person:
    def __init__(self, first_name):
        self._first_name = first_name

    # Getter function
    @property
    def first_name(self):
        return self._first_name

    # Setter function
    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value

    # Deleter function (optional)
    @first_name.deleter
    def first_name(self):
        raise AttributeError("Can't delete attribute")

# 调用父类方法

In [16]:
class Base:
    def __init__(self):
        print('Base.__init__')

class A(Base):
    def __init__(self):
        super().__init__()
        print('A.__init__')

class B(Base):
    def __init__(self):
        super().__init__()
        print('B.__init__')

class C(A,B):
    def __init__(self):
        super().__init__()  # Only one call to super() here
        print('C.__init__')

In [17]:
C.__mro__

(__main__.C, __main__.A, __main__.B, __main__.Base, object)

In [18]:
c = C()

Base.__init__
B.__init__
A.__init__
C.__init__


# 子类中扩展property

In [29]:
class Person:
    def __init__(self, name):
        self.name = name

    # Getter function
    @property
    def name(self):
        return self._name

    # Setter function
    @name.setter
    def name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._name = value

    # Deleter function
    @name.deleter
    def name(self):
        raise AttributeError("Can't delete attribute")

In [32]:
class SubPerson(Person):
    @Person.name.setter
    def name(self, value):
        print('Setting name to', value)
        super(SubPerson, SubPerson).name.__set__(self,value) # 使用类变量访问

In [33]:
s = SubPerson('Guido')

Setting name to Guido


In [28]:
s.name

Getting name


'Guido'

In [34]:
# 创建新的类或实例属性

In [ ]:
# Descriptor for a type-checked attribute
class Typed:
    def __init__(self, name, expected_type):
        self.name = name
        self.expected_type = expected_type
    def __get__(self, instance, cls):
        if instance is None:
            # 访问类属性时instance=None
            return self
        else:
            return instance.__dict__[self.name]

    def __set__(self, instance, value):
        if not isinstance(value, self.expected_type):
            raise TypeError('Expected ' + str(self.expected_type))
        instance.__dict__[self.name] = value
    def __delete__(self, instance):
        del instance.__dict__[self.name]

# Class decorator that applies it to selected attributes
def typeassert(**kwargs):
    def decorate(cls):
        for name, expected_type in kwargs.items():
            # Attach a Typed descriptor to the class
            setattr(cls, name, Typed(name, expected_type))
        return cls
    return decorate

# Example use
@typeassert(name=str, shares=int, price=float)
class Stock:
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price